In [2]:
import pandas as pd 

df = pd.read_csv('midas_time.csv',low_memory=False)

In [6]:
source = pd.read_csv('CodeJIT_Dataset_Commit_Hash.csv')['commit_id']
source = list(source)

df = df[df['commit_id'].isin(source)]

In [8]:
len(set(df['commit_id']))

20274

In [5]:
sample = df.sample(10)
sample.to_csv('sample_data.csv')

In [17]:
df.head(1)

,commit_id,diff,label,filename,LOC_MOD,partition,repo,PL
31194,c5759e7b76f5bf844be6c6641cc1b356bbc83869,"@@ -41,6 +41,7 @@ OF THIS SOFTWARE, EVEN IF AD...",1,Esm/ib/src/linux/fm_cmd/fm_cmd.c,34,train,01org___opa-fm,java


In [21]:
from tqdm import tqdm
def get_change(diff):
    res = list()
    for l in diff.splitlines():
        if l.startswith('+') or l.startswith('-'):
            res.append(l)
    return '\n'.join(res)

dict_diff = dict()
dict_only_change = dict()
for cm in tqdm(source):
    tmp_df = df[df['commit_id'] == cm]
    diff_str = list()
    change_str = list()
    for _,row in tmp_df.iterrows():
        diff_str.append(row['diff'])
        change_str.append(get_change(row['diff']))
    # [print(el) for el in diff_str]
    # print('*'*30)
    # [print(el) for el in change_str]
    dict_diff[cm] = diff_str
    dict_only_change[cm] = change_str
    # break

100%|██████████| 20274/20274 [02:08<00:00, 158.01it/s]


In [32]:
import json

with open('diff.json','w+') as f:
    json.dump(dict_diff,f)
with open('only_change.json','w+') as f:
    json.dump(dict_only_change,f)

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")
model.to(device)

In [31]:
idx_diff = dict()
for cm,diff in tqdm(dict_diff.items()):
    diff_text = f' {tokenizer.sep_token} '.join(diff)
    encode = tokenizer.encode_plus(
            diff_text,
            truncation=True,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
        )
    idx_diff[cm] = encode
    

100%|██████████| 20274/20274 [07:31<00:00, 44.88it/s] 


In [36]:
# with open('index_diff.json','w+') as f:
#     json.dump(idx_diff,f)
import pickle
pickle.dump(idx_diff,open('index_diff.pkl','wb'),protocol=3)

In [51]:
for cm,encode in tqdm(idx_diff.items()):
    encode.to(device)
    embedd = model(input_ids=encode.input_ids,attention_mask=encode.attention_mask)[0][:,0,:]
    torch.save(embedd,f'diff/{cm}.pt',)
    # break

100%|██████████| 20274/20274 [12:10<00:00, 27.74it/s]


In [52]:
idx_only_change = dict()
for cm,diff in tqdm(dict_only_change.items()):
    diff_text = f' {tokenizer.sep_token} '.join(diff)
    encode = tokenizer.encode_plus(
            diff_text,
            truncation=True,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
        )
    idx_only_change[cm] = encode
import pickle
pickle.dump(idx_only_change,open('index_only_change.pkl','wb'),protocol=3)

100%|██████████| 20274/20274 [05:02<00:00, 67.12it/s] 


In [53]:
for cm,encode in tqdm(idx_only_change.items()):
    encode.to(device)
    embedd = model(input_ids=encode.input_ids,attention_mask=encode.attention_mask)[0][:,0,:]
    torch.save(embedd,f'changes/{cm}.pt',)
    # break

100%|██████████| 20274/20274 [12:16<00:00, 27.51it/s]


In [54]:
!zip -r diff.zip diff 
!zip -r changes.zip changes

  adding: diff/ (stored 0%)
  adding: diff/0bfacb95dea6a04e4c10a24b9d90c14a401ade67.pt (deflated 63%)
  adding: diff/660a2d83e026496db6b3eaec2256a2cdd6c74de8.pt (deflated 74%)
  adding: diff/4859d69e07504d4b0a4bdf9bcb4d9e3769ca35ae.pt (deflated 7%)
  adding: diff/6f8bb34c2993a48ab9afbe353e6d0cff7c8d821d.pt (deflated 7%)
  adding: diff/baba2eedacbbaecf55bdb89dbfe32c69799df99f.pt (deflated 7%)
  adding: diff/a59505ca76718549dfc51b9622e2d88cb60f33b5.pt (deflated 74%)
  adding: diff/3426d575bf46edc0f52d15f7e7c1d199e8688faa.pt (deflated 49%)
  adding: diff/afd9096eb1882f23929f5b5c177898ed231bac66.pt (deflated 61%)
  adding: diff/9fbf0fa81fca8f527669dd4fa72662d66e7d6329.pt (deflated 7%)
  adding: diff/c81131db15dd1844d0db1d51f3cd7a105cfd2cf3.pt (deflated 7%)
  adding: diff/7e80b63ecd259d69d383623e75b318bf2bd491f6.pt (deflated 62%)
  adding: diff/31bed5509dfcbdfc293154ce81086a4dbd7a80b6.pt (deflated 59%)
  adding: diff/347cb14b7cba7560e53f4434b419b9d8800253e7.pt (deflated 64%)
  adding: diff/